In [1]:
%matplotlib notebook
from ipywidgets import *
from math import sqrt
import numpy as np
import matplotlib.pyplot as plt
import os.path
import re
import json
import numpy as np
from numpy import genfromtxt
import pandas as pd
from keras.layers import Dense, Input, average
from keras.layers import LSTM
from keras.optimizers import Adam
from keras.models import Sequential
from sklearn.metrics import mean_absolute_error
from keras.layers import Dropout, Flatten,Conv1D
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from keras.models import Model
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
from keras.models import load_model
import tensorflow as tf
from pmdarima.arima import auto_arima
from sklearn import preprocessing
from rpy2.robjects.packages import STAP
import math
import uber
import uber2
import time
tf.compat.v1.logging.set_verbosity(tf.logging.ERROR)
import rpy2
from rpy2.rinterface_lib.callbacks import logger as rpy2_logger
import logging
rpy2_logger.setLevel(logging.ERROR)   # will display errors, but not warnings
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

Using TensorFlow backend.
W0714 20:38:40.009419 140343950468928 deprecation_wrapper.py:119] From /home/jonas/GuidedRes/uber.py:30: The name tf.logging.set_verbosity is deprecated. Please use tf.compat.v1.logging.set_verbosity instead.

W0714 20:38:40.011229 140343950468928 deprecation_wrapper.py:119] From /home/jonas/GuidedRes/uber.py:30: The name tf.logging.ERROR is deprecated. Please use tf.compat.v1.logging.ERROR instead.



In [2]:
import keras
print(keras.__version__)
print(tf.__version__)

2.2.4
1.14.0


In [3]:
DATASET_STATES_CSV = 'States_Timeseries.csv'
FORECASTING_STEPS = 6
RANDOM_SEED = 3

SCALER_STANDARD = preprocessing.StandardScaler
SCALER_MINMAX = preprocessing.MinMaxScaler
DATA_SCALER = SCALER_MINMAX


In [4]:
def normalize_df(df,scaler):
    x = df.values #returns a numpy array
    x_train = df.values[:-6]
    display(x_train.shape)
    scaler = scaler.fit(x_train)
    x_scaled = scaler.transform(x)
    df = pd.DataFrame(x_scaled, columns=df.columns, index=df.index)
    return df


def normalizeInputTouple(x,y,scaler):
    x = np.array(x)
    x = np.reshape(x,(-1,1))
    x = scaler.fit_transform(x)
    x = x[:,0]
    y = y.reshape(-1,1)
    y = scaler.transform(y)
    y = y[:,0]
    return x,y

def getTrainTestSplit(series):
    series_np = np.array(series)
    return series_np[:-FORECASTING_STEPS],series_np[-FORECASTING_STEPS:]

def reshapeInputs(data):
    data = np.array(data)
    return np.reshape(data, (data.shape[0], data.shape[1],1))

def create_dataset_scaled(dataset,scaler,LOOK_BACK):
    dataX, dataY = [], []
    for i in range(0,len(dataset) - LOOK_BACK -FORECASTING_STEPS+1, int(LOOK_BACK/2)):
        x = dataset[i:(i + LOOK_BACK)]
        y = dataset[i + LOOK_BACK:i+LOOK_BACK+FORECASTING_STEPS]
        normalizeInputTouple(x,y,scaler)
        dataX.append(x)
        dataY.append(y)
    return reshapeInputs(dataX), np.array(dataY)



def evaluateModel(model,x,y,scaler,wsc = False):
    prediction = model.predict(x)
    if not scaler == None:
        prediction = scaler.inverse_transform(prediction)
        y = scaler.inverse_transform(y)
    score = mean_squared_error(y.ravel(),prediction.ravel())
    return score

class NotInitialized(Exception):
     def __init__(self, value):
        self.value = value
     def __str__(self):
         return repr(self.value)

def getModel_vLSTM(config):
    window = config['window'][0]
    model = Sequential()
    return_sequences = True if config['layers'][0]==3 else False
    model.add(LSTM(config['cs1'][0], activation='tanh',input_shape=(window, 1),return_sequences = True))
    model.add(LSTM(config['cs2'][0], activation='tanh',return_sequences=return_sequences))
    if config['layers'][0]==3:
         model.add(LSTM(config['cs3'][0], activation='tanh'))
    model.add(Dropout(config['dropout'][0]))
    model.add(Dense(FORECASTING_STEPS, activation='sigmoid'))
    lr = math.pow(10,-config['lr'][0])
    opt = Adam(lr=lr, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
    model.compile(loss='mse', optimizer=opt,  metrics=['mean_absolute_error'])
    return model
MODEL = None

def initLSTM(train_all,rep):
    global MODEL
    if True:#not os.path.isfile('Models/LSTMALL_'+str(rep)+'.h5'):
        scaler = SCALER_STANDARD()
        EPOCHS = 50
        BATCH_SIZE =1
        config = {'window':[3],'layers':[2],'cs1':[4],'cs2':[4],'cs3':[32],'dropout':[0.0],'lr':[1]}
        #config = {'window':[12],'layers':[2],'cs1':[16],'cs2':[8],'cs3':[15],'dropout':[0],'lr':[3]}
        look_back = config['window'][0]
        model = getModel_vLSTM(config)
        x_train,y_train = create_dataset_scaled(train_all[0],scaler,look_back)
        for train in train_all[1:]:
            x_train_h,y_train_h = create_dataset_scaled(train,scaler,look_back)
            x_train = np.vstack((x_train,x_train_h))
            y_train = np.vstack((y_train,y_train_h))
        model.fit(x_train, y_train, epochs=EPOCHS, batch_size=BATCH_SIZE, verbose=0)
        model.save('Models/LSTMALL_'+str(rep)+'.h5')
        MODEL = model

def evaluateLSTM(train,test,state,rep):
    global MODEL
    scaler = SCALER_STANDARD()
    EPOCHS = 50
    BATCH_SIZE =1
    look_back = 3
    x_train,y_train = create_dataset_scaled(train,scaler,look_back)
    model = MODEL
    if MODEL == None and os.path.isfile('Models/LSTMALL_'+str(rep)+'.h5'):
        model = load_model('Models/LSTMALL_'+str(rep)+'.h5')
    elif MODEL == None:
        raise NotInitialized("Please call initLSTM before calling evaluateLSTM")
    x_test =train[-look_back:]
    y_test = test
    x_test,y_test = normalizeInputTouple(x_test,y_test,scaler)
    x_test = reshapeInputs([x_test])
    y_test = np.array([y_test])
    
    return evaluateModel(model,x_test,y_test,scaler,True)




In [5]:
df = pd.read_csv(DATASET_STATES_CSV,index_col=0,parse_dates=True,encoding = "utf-8")
df = normalize_df(df,DATA_SCALER())

(67, 16)

In [6]:
Final_Results = {}
for rep in range(0,10):
    overall_test_scores = {}
    nbr_of_states = len(list(df))
    approaches = []
    approaches.append({'name':'LSTM', 'func': evaluateLSTM})
    start = time.time()
    state_nr = 0
    
    training_data = []
    for state in df:
        series = df[state]
        x,y = getTrainTestSplit(series)
        training_data.append(x)
    initLSTM(training_data,rep)
    
    for state in df:
        print(f'{state}:')
        for approach in approaches:
            series = df[state]
            x,y = getTrainTestSplit(series)
            test_score = approach['func'](x,y,state_nr,rep)
            prevStateDict = Final_Results.get(state,{})
            prevRes = prevStateDict.get(approach['name'],[])
            prevRes.append(test_score)
            prevStateDict[approach['name']] = prevRes
            Final_Results[state] = prevStateDict
            print(approach['name']+'_sqrt:'+ str(sqrt(test_score)))
            overall_test_scores[approach['name']]= overall_test_scores.get(approach['name'],0) + test_score
        state_nr = state_nr +1
    print('AVG:')
    for approach in approaches:
        overall_test_scores[approach['name']] = overall_test_scores[approach['name']]/nbr_of_states
        print(approach['name']+'_sqrt:'+str(sqrt(overall_test_scores[approach['name']])))
    end = time.time()
    print(f'{(end - start)/60} minutes runtime')

with open('my_Final_Res_NN_All_LSTM.json', 'w') as f:
    json.dump(Final_Results, f)

Schleswig-Holstein:
LSTM_sqrt:0.18713772216923164
Freie und Hansestadt Hamburg:
LSTM_sqrt:0.11720417103557576
Niedersachsen:
LSTM_sqrt:0.12211510347816072
Freie Hansestadt Bremen:
LSTM_sqrt:0.26581042541120636
Nordrhein-Westfalen:
LSTM_sqrt:0.10788915109189083
Hessen:
LSTM_sqrt:0.14996829484193677
Rheinland-Pfalz:
LSTM_sqrt:0.17865016609339066
Baden-Württemberg:
LSTM_sqrt:0.08029065383480974
Freistaat Bayern:
LSTM_sqrt:0.15270970101207432
Saarland:
LSTM_sqrt:0.16437310924955542
Berlin:
LSTM_sqrt:0.17542440559886957
Brandenburg:
LSTM_sqrt:0.09519463552420904
Mecklenburg-Vorpommern:
LSTM_sqrt:0.1499554750211938
Freistaat Sachsen:
LSTM_sqrt:0.07498624453280926
Sachsen-Anhalt:
LSTM_sqrt:0.10902768376421339
Freistaat Thüringen:
LSTM_sqrt:0.0943688382147106
AVG:
LSTM_sqrt:0.14703194002323833
8.561509243647258 minutes runtime
Schleswig-Holstein:
LSTM_sqrt:0.18814525923362532
Freie und Hansestadt Hamburg:
LSTM_sqrt:0.1161941862174372
Niedersachsen:
LSTM_sqrt:0.12068166683976278
Freie Hansestad

In [7]:
display(Final_Results)

{'Schleswig-Holstein': {'LSTM': [0.03502052705868853,
   0.035398638572088074,
   0.034827476349536375,
   0.03506236461296336,
   0.03553746355036275,
   0.03600461033431465,
   0.034403554026539045,
   0.035334142386531896,
   0.03537472934536811,
   0.035321778782616525]},
 'Freie und Hansestadt Hamburg': {'LSTM': [0.013736817708136497,
   0.013501088910732472,
   0.013217233229710927,
   0.01274385895924162,
   0.01359817475985,
   0.013667232668996549,
   0.01317554801284154,
   0.01375847004904115,
   0.01451592718910503,
   0.01330800332398876]},
 'Niedersachsen': {'LSTM': [0.014912098497481902,
   0.014564064711223498,
   0.014319530159163936,
   0.013590977937963987,
   0.01467292661286993,
   0.014645054520900652,
   0.014410912777979285,
   0.01485651901514616,
   0.015536999173813149,
   0.014397192398649622]},
 'Freie Hansestadt Bremen': {'LSTM': [0.07065518225728651,
   0.06621575671677997,
   0.06633938844597778,
   0.06028762539568974,
   0.06736348721201006,
   0.06549